## Solve Model balancing problem with kinetic parameters as variables

In [1]:
import numpy as np
from model_balancing import ModelBalancing, Q_
import cvxpy as cp

In [2]:
use_regulation = True
args = {}

# choose solver

if cp.MOSEK in cp.installed_solvers():
    args["solver"] = cp.MOSEK
else:
    #args["solver"] = cp.SCS
    args["solver"] = cp.ECOS

In [3]:
# Network topology

args["S"] = np.matrix(
    [
        [-1, 0, 0],
        [1, -1, -1],
        [0, 1, 1]
    ], dtype=float
)

if use_regulation:
    args["A_act"] = np.matrix(
        [
            [0, 1, 1],
            [0, 0, 0],
            [0, 0, 0]
        ], dtype=float
    )
    args["A_inh"] = np.matrix(
        [
            [0, 0, 0],
            [0, 0, 0],
            [1, 0, 0]
        ], dtype=float
    )
else:
    args["A_act"] = np.matrix(
        [
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0]
        ], dtype=float
    )
    args["A_inh"] = np.matrix(
        [
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0]
        ], dtype=float
    )

In [4]:
# Priors for Kinetic and Thermodynamic parameters

args["Keq_gmean"] = Q_([1, 1, 1], "")  # geometric mean (assuming a standard concentration of 1M)
args["Keq_ln_cov"] = np.array(
    [
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1],
    ]
)  # log-scale covariance

args["kcatf_gmean"] = Q_([20.0, 8.0, 10.0], "1/s")  # geometric mean
args["kcatf_ln_cov"] = np.array(
    [
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1],
    ]
) # log-scale covariance

args["kcatr_gmean"] = Q_([3.5e-3, 6.2e4, 3.1e-1], "1/s")  # geometric mean
args["kcatr_ln_cov"] = np.array(
    [
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1],
    ]
) # log-scale covariance

# the K parameters are only for existing connections in the S, A_act and A_inh matrices.
# the order of values is "metabolite-first".
args["Km_gmean"] = Q_([1e-2, 1e-4, 1e-4, 1e-3, 1e-1, 1e-1], "M")
args["Km_ln_cov"] = np.array(
    [
        [1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1],
    ]
) # log-scale covariance

args["Ka_gmean"] = Q_([1e-3, 5e-3], "M")
args["Ka_ln_cov"] = np.array(
    [
        [1, 0],
        [0, 1],
    ]
)

args["Ki_gmean"] = Q_([1e-3], "M")
args["Ki_ln_cov"] = np.array(
    [
        [1],
    ]
)

In [5]:
# condition dependent data (columns represent conditions)

args["fluxes"] = Q_(
    [
        [2.0, 1.5, 2.5, 2.5],
        [1.0, 1.0, 2.0, 0.5],
        [1.0, 0.5, 0.5, 2.0],
    ], "mM/s"
)

args["conc_enz_gmean"] = Q_(
    [
        [1e-3, 1e-3, 1e-3, 3e-3],
        [2e-3, 3e-3, 1e-3, 2e-3],
        [1e-3, 2e-3, 2e-3, 1e-3],
    ], "M")  # geometric mean

args["conc_enz_gstd"] = np.array(
    [
        [1.1, 1.1, 1.1, 1.1],
        [1.1, 1.1, 1.1, 1.1],
        [1.1, 1.1, 1.1, 1.1],
    ]
)  # geometric standard deviation

args["conc_met_gmean"] = Q_(
    [
        [3e-3, 4e-3, 9e-4, 3e-4],
        [2e-3, 3e-3, 6e-4, 2e-4],
        [1e-3, 2e-3, 3e-4, 1e-4],
    ], "M"
)  # geometric mean

args["conc_met_gstd"] = np.array(
    [
        [1.1, 1.1, 1.1, 1.1],
        [1.1, 1.1, 1.1, 1.1],
        [1.1, 1.1, 1.1, 1.1],
    ]
)  # geometric standard deviation

In [6]:
args["rate_law"] = "CM"

mb = ModelBalancing(**args)

# check that the driving forces at the mode are all positive
assert mb.is_gmean_feasible()

mb.initialize_with_gmeans()
print(f"initial total squared Z-scores = {mb.objective_value:.3f}")

mb.solve()
print(f"optimized total squared Z-scores = {mb.objective_value:.3f}")

print("-"*25 + " All Z-scores " + "-"*25)
mb.print_z_scores()

initial total squared Z-scores = 0.000
optimized total squared Z-scores = 0.629
------------------------- All Z-scores -------------------------
enzymes =  0.0
metabolites =  0.01
Keq =  0.23
kcat forward =  0.16
kcat reverse =  0.01
Km =  0.05
Ka =  0.14
Ki =  0.03


In [7]:
mb.print_status()


Metabolite concentrations (M) =
 [[3.00000000e-03 4.00213146e-03 9.02560533e-04 3.01428562e-04]
 [2.00654461e-03 3.00462420e-03 6.00000000e-04 1.99228363e-04]
 [9.97869880e-04 1.99381977e-03 3.00000000e-04 1.00000000e-04]]

Enzyme concentrations (M) =
 [[0.01612117 0.02177984 0.01517331 0.01693491]
 [0.00026595 0.00032576 0.00093723 0.00061825]
 [0.00035917 0.00017371 0.0007039  0.01618507]]

Driving forces (RT) =
 [[0.74651355 0.63098999 0.75262151 0.75839674]
 [0.85707379 0.56862748 0.85167444 0.84780879]
 [0.98970921 0.70126289 0.98430985 0.9804442 ]]

η(thr) =
 [[0.53 0.47 0.53 0.53]
 [0.58 0.43 0.57 0.57]
 [0.63 0.5  0.63 0.62]]

η(kin) =
 [[0.02 0.02 0.02 0.01]
 [0.95 0.97 0.86 0.67]
 [0.68 0.76 0.39 0.18]]

η(reg) =
 [[0.54 0.37 0.8  0.92]
 [0.75 0.8  0.47 0.23]
 [0.47 0.54 0.21 0.08]]




